In [1]:
import numpy as np
import torch
import pdb
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import struct
import gzip
from PIL import Image
def load_data_image():
    #读取文件
    binfile = open('../data/train-images.idx3-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前四个无符号整数
    magic, num, cows, cols = struct.unpack_from('>IIII',buffers,0)
    #获得偏移
    offset = struct.calcsize('>IIII')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num*cows*cols) + 'B'
    #读取图片数据并转化矩阵
    training_image = struct.unpack_from(strString, buffers, offset)
    training_image = np.reshape(training_image,(num,cows*cols))
    binfile.close()
    return (training_image)
def load_data_lable():
    #读取文件
    binfile = open('../data/train-labels.idx1-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前两个无符号整数
    magic, num = struct.unpack_from('>II',buffers,0)
    #获得偏移
    offset = struct.calcsize('>II')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num) + 'B'
    #读取图片数据并转化矩阵
    training_lable = struct.unpack_from(strString, buffers, offset)
    training_lable = np.reshape(training_lable,(num))
    binfile.close()
    return (training_lable)
def I(z):#S型神经元
    for x in range(784):
        z[x] = z[x]>0
    return z
training_image = load_data_image()
training_lable = load_data_lable()
training_lables = np.zeros((60000,10))

for n,x in zip(range(60000),training_lable):
    training_lables[n][x] = 1
training_lable = training_lables

for x,i in zip(training_image,range(60000)):
    training_image[i] = I(x)
    
test_image = training_image[50000:]
test_lable = training_lable[50000:]
training_image = training_image[:50000]
training_lable = training_lable[:50000]
training_data = zip(training_image,training_lable)
test_data = zip(test_image,test_lable)
training_data = list(training_data)
test_data = list(test_data)

In [6]:
print(training_lable.shape)

(50000, 10)


In [4]:
import pdb
class Network(object):
    def __init__(self,size):
        self.model = torch.nn.Sequential(
            torch.nn.Linear(size[0], size[1], bias = False), # w_1 * x +b_1
            torch.nn.ReLU(),
            torch.nn.Linear(size[1], size[2] , bias = False),
            torch.nn.ReLU(),
            torch.nn.Linear(size[2], size[3] , bias = False),
            torch.nn.Sigmoid(),
        )
        self.loss_fn = nn.MSELoss(reduction = 'sum')
        
    def run(self,training_data,epochs,mini_batch_size,eta,test_data = None):
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = eta)
        
        if test_data: 
            test_data = list(test_data)
            test_data = test_data[:1000]
            n_test = len(test_data)
        training_data = list(training_data)
        
#         training_data = training_data[:1000] #可以改变训练集大小，测试使用
        
        n = len(training_data)
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            k = len(mini_batches)
            for  mini_batch in mini_batches:
                loss = self.training(mini_batch,mini_batch_size)
            print('LOSS {0},'.format(loss))
            if j >= 0:
                print('Epoch {0}:{1} / {2}'. format(
                        j, self.test(test_data), n_test))
    def training(self, training_data, size):
        x = torch.zeros((size,784),dtype = torch.float)
        y = torch.zeros((size,10), dtype = torch.float)
        for (a, b), i in zip(training_data, range(size)):
            x[i] = torch.from_numpy(a) #转换成二维的矩阵，后面做乘法方便
            y[i] = torch.from_numpy(b)
        #Forward pass
        y_pred = self.model(x) # model.forward()
        #loss
        loss = self.loss_fn(y_pred , y) #computation graph  
        #重置参数增加量
        self.optimizer.zero_grad()
        # Backward pass
        loss.backward()
            
        #update model parameters
        self.optimizer.step(),
        
        return loss.item()
    def test(self,test_data):
        test_results = [(torch.argmax(self.model(torch.from_numpy(x).reshape(1,784).float())), torch.from_numpy(y))
                for x, y in test_data]
        return sum(y[int(x)] == 1 for (x, y) in test_results)

In [3]:
net = Network([784,100,50,10])
net.run(training_data,10,5,0.0001,test_data)

tensor(12.5598, grad_fn=<MseLossBackward>)
tensor(12.5132, grad_fn=<MseLossBackward>)
tensor(12.4247, grad_fn=<MseLossBackward>)
tensor(12.3530, grad_fn=<MseLossBackward>)
tensor(12.3978, grad_fn=<MseLossBackward>)
tensor(12.4083, grad_fn=<MseLossBackward>)
tensor(12.3919, grad_fn=<MseLossBackward>)
tensor(12.2615, grad_fn=<MseLossBackward>)
tensor(12.3555, grad_fn=<MseLossBackward>)
tensor(12.3202, grad_fn=<MseLossBackward>)
tensor(12.2849, grad_fn=<MseLossBackward>)
tensor(12.1807, grad_fn=<MseLossBackward>)
tensor(12.0719, grad_fn=<MseLossBackward>)
tensor(12.1447, grad_fn=<MseLossBackward>)
tensor(12.0071, grad_fn=<MseLossBackward>)
tensor(11.9807, grad_fn=<MseLossBackward>)
tensor(12.0000, grad_fn=<MseLossBackward>)
tensor(11.9861, grad_fn=<MseLossBackward>)
tensor(11.6862, grad_fn=<MseLossBackward>)
tensor(11.7956, grad_fn=<MseLossBackward>)
tensor(11.8699, grad_fn=<MseLossBackward>)
tensor(11.8154, grad_fn=<MseLossBackward>)
tensor(11.7617, grad_fn=<MseLossBackward>)
tensor(11.6

tensor(4.7062, grad_fn=<MseLossBackward>)
tensor(4.2651, grad_fn=<MseLossBackward>)
tensor(4.6391, grad_fn=<MseLossBackward>)
tensor(4.5771, grad_fn=<MseLossBackward>)
tensor(4.4049, grad_fn=<MseLossBackward>)
tensor(4.7123, grad_fn=<MseLossBackward>)
tensor(4.8124, grad_fn=<MseLossBackward>)
tensor(4.3334, grad_fn=<MseLossBackward>)
tensor(4.6381, grad_fn=<MseLossBackward>)
tensor(4.8376, grad_fn=<MseLossBackward>)
tensor(4.7554, grad_fn=<MseLossBackward>)
tensor(4.5567, grad_fn=<MseLossBackward>)
tensor(4.7326, grad_fn=<MseLossBackward>)
tensor(4.6288, grad_fn=<MseLossBackward>)
tensor(4.4074, grad_fn=<MseLossBackward>)
tensor(4.4296, grad_fn=<MseLossBackward>)
tensor(4.7237, grad_fn=<MseLossBackward>)
tensor(4.7807, grad_fn=<MseLossBackward>)
tensor(4.0484, grad_fn=<MseLossBackward>)
tensor(4.9585, grad_fn=<MseLossBackward>)
tensor(4.6635, grad_fn=<MseLossBackward>)
tensor(4.5747, grad_fn=<MseLossBackward>)
tensor(4.5637, grad_fn=<MseLossBackward>)
tensor(4.1537, grad_fn=<MseLossBac

tensor(4.3199, grad_fn=<MseLossBackward>)
tensor(4.4975, grad_fn=<MseLossBackward>)
tensor(3.9449, grad_fn=<MseLossBackward>)
tensor(4.0855, grad_fn=<MseLossBackward>)
tensor(4.0575, grad_fn=<MseLossBackward>)
tensor(4.2433, grad_fn=<MseLossBackward>)
tensor(4.4984, grad_fn=<MseLossBackward>)
tensor(4.4633, grad_fn=<MseLossBackward>)
tensor(4.4539, grad_fn=<MseLossBackward>)
tensor(4.5546, grad_fn=<MseLossBackward>)
tensor(4.2945, grad_fn=<MseLossBackward>)
tensor(3.8583, grad_fn=<MseLossBackward>)
tensor(4.1272, grad_fn=<MseLossBackward>)
tensor(4.1574, grad_fn=<MseLossBackward>)
tensor(4.0723, grad_fn=<MseLossBackward>)
tensor(4.2953, grad_fn=<MseLossBackward>)
tensor(4.0038, grad_fn=<MseLossBackward>)
tensor(4.4043, grad_fn=<MseLossBackward>)
tensor(4.2807, grad_fn=<MseLossBackward>)
tensor(4.0675, grad_fn=<MseLossBackward>)
tensor(4.3378, grad_fn=<MseLossBackward>)
tensor(4.3105, grad_fn=<MseLossBackward>)
tensor(3.8071, grad_fn=<MseLossBackward>)
tensor(3.9428, grad_fn=<MseLossBac

tensor(3.5112, grad_fn=<MseLossBackward>)
tensor(2.5595, grad_fn=<MseLossBackward>)
tensor(3.2565, grad_fn=<MseLossBackward>)
tensor(3.7561, grad_fn=<MseLossBackward>)
tensor(3.6921, grad_fn=<MseLossBackward>)
tensor(4.0214, grad_fn=<MseLossBackward>)
tensor(3.3641, grad_fn=<MseLossBackward>)
tensor(3.9781, grad_fn=<MseLossBackward>)
tensor(3.2778, grad_fn=<MseLossBackward>)
tensor(3.4804, grad_fn=<MseLossBackward>)
tensor(4.0369, grad_fn=<MseLossBackward>)
tensor(3.4009, grad_fn=<MseLossBackward>)
tensor(4.2873, grad_fn=<MseLossBackward>)
tensor(3.0447, grad_fn=<MseLossBackward>)
tensor(4.3262, grad_fn=<MseLossBackward>)
tensor(3.6956, grad_fn=<MseLossBackward>)
tensor(3.2874, grad_fn=<MseLossBackward>)
tensor(2.8054, grad_fn=<MseLossBackward>)
tensor(3.2028, grad_fn=<MseLossBackward>)
tensor(3.1852, grad_fn=<MseLossBackward>)
tensor(2.2303, grad_fn=<MseLossBackward>)
tensor(3.3799, grad_fn=<MseLossBackward>)
tensor(4.3205, grad_fn=<MseLossBackward>)
tensor(4.0304, grad_fn=<MseLossBac

tensor(2.2214, grad_fn=<MseLossBackward>)
tensor(2.9671, grad_fn=<MseLossBackward>)
tensor(3.2694, grad_fn=<MseLossBackward>)
tensor(2.4124, grad_fn=<MseLossBackward>)
tensor(2.5234, grad_fn=<MseLossBackward>)
tensor(4.2996, grad_fn=<MseLossBackward>)
tensor(3.9791, grad_fn=<MseLossBackward>)
tensor(2.5682, grad_fn=<MseLossBackward>)
tensor(2.5548, grad_fn=<MseLossBackward>)
tensor(2.6786, grad_fn=<MseLossBackward>)
tensor(2.8562, grad_fn=<MseLossBackward>)
tensor(2.3103, grad_fn=<MseLossBackward>)
tensor(2.8849, grad_fn=<MseLossBackward>)
tensor(3.0787, grad_fn=<MseLossBackward>)
tensor(2.5287, grad_fn=<MseLossBackward>)
tensor(2.8034, grad_fn=<MseLossBackward>)
tensor(2.5267, grad_fn=<MseLossBackward>)
tensor(2.6658, grad_fn=<MseLossBackward>)
tensor(2.7735, grad_fn=<MseLossBackward>)
tensor(3.4267, grad_fn=<MseLossBackward>)
tensor(2.2396, grad_fn=<MseLossBackward>)
tensor(3.0645, grad_fn=<MseLossBackward>)
tensor(3.4539, grad_fn=<MseLossBackward>)
tensor(3.1965, grad_fn=<MseLossBac

...

tensor(0.5672, grad_fn=<MseLossBackward>)
tensor(0.7796, grad_fn=<MseLossBackward>)
tensor(1.5665, grad_fn=<MseLossBackward>)
tensor(1.9278, grad_fn=<MseLossBackward>)
tensor(1.2656, grad_fn=<MseLossBackward>)
tensor(2.4896, grad_fn=<MseLossBackward>)
tensor(0.3898, grad_fn=<MseLossBackward>)
tensor(0.1072, grad_fn=<MseLossBackward>)
tensor(0.8839, grad_fn=<MseLossBackward>)
tensor(0.1683, grad_fn=<MseLossBackward>)
tensor(1.4783, grad_fn=<MseLossBackward>)
tensor(0.1467, grad_fn=<MseLossBackward>)
tensor(0.2807, grad_fn=<MseLossBackward>)
tensor(1.0437, grad_fn=<MseLossBackward>)
tensor(0.2853, grad_fn=<MseLossBackward>)
tensor(0.9020, grad_fn=<MseLossBackward>)
tensor(1.7179, grad_fn=<MseLossBackward>)
tensor(1.6259, grad_fn=<MseLossBackward>)
tensor(1.1745, grad_fn=<MseLossBackward>)
tensor(0.7784, grad_fn=<MseLossBackward>)
tensor(1.6037, grad_fn=<MseLossBackward>)
tensor(1.5470, grad_fn=<MseLossBackward>)
tensor(0.2133, grad_fn=<MseLossBackward>)
tensor(0.2237, grad_fn=<MseLossBac

tensor(2.3762, grad_fn=<MseLossBackward>)
tensor(1.5107, grad_fn=<MseLossBackward>)
tensor(1.4363, grad_fn=<MseLossBackward>)
tensor(0.0941, grad_fn=<MseLossBackward>)
tensor(2.8349, grad_fn=<MseLossBackward>)
tensor(0.5607, grad_fn=<MseLossBackward>)
tensor(0.6471, grad_fn=<MseLossBackward>)
tensor(1.6910, grad_fn=<MseLossBackward>)
tensor(1.4306, grad_fn=<MseLossBackward>)
tensor(0.8548, grad_fn=<MseLossBackward>)
tensor(1.1760, grad_fn=<MseLossBackward>)
tensor(1.0706, grad_fn=<MseLossBackward>)
tensor(1.3024, grad_fn=<MseLossBackward>)
tensor(2.8630, grad_fn=<MseLossBackward>)
tensor(1.7776, grad_fn=<MseLossBackward>)
tensor(0.2887, grad_fn=<MseLossBackward>)
tensor(0.4497, grad_fn=<MseLossBackward>)
tensor(0.4382, grad_fn=<MseLossBackward>)
tensor(3.0920, grad_fn=<MseLossBackward>)
tensor(1.0340, grad_fn=<MseLossBackward>)
tensor(0.2467, grad_fn=<MseLossBackward>)
tensor(0.1473, grad_fn=<MseLossBackward>)
tensor(1.9597, grad_fn=<MseLossBackward>)
tensor(1.0688, grad_fn=<MseLossBac

tensor(0.0724, grad_fn=<MseLossBackward>)
tensor(1.0802, grad_fn=<MseLossBackward>)
tensor(1.9696, grad_fn=<MseLossBackward>)
tensor(0.1859, grad_fn=<MseLossBackward>)
tensor(1.6223, grad_fn=<MseLossBackward>)
tensor(0.2183, grad_fn=<MseLossBackward>)
tensor(2.0308, grad_fn=<MseLossBackward>)
tensor(1.2714, grad_fn=<MseLossBackward>)
tensor(1.9017, grad_fn=<MseLossBackward>)
tensor(0.1670, grad_fn=<MseLossBackward>)
tensor(3.1031, grad_fn=<MseLossBackward>)
tensor(0.3217, grad_fn=<MseLossBackward>)
tensor(0.7592, grad_fn=<MseLossBackward>)
tensor(0.2983, grad_fn=<MseLossBackward>)
tensor(0.1690, grad_fn=<MseLossBackward>)
tensor(1.2422, grad_fn=<MseLossBackward>)
tensor(1.1844, grad_fn=<MseLossBackward>)
tensor(0.3540, grad_fn=<MseLossBackward>)
tensor(0.1050, grad_fn=<MseLossBackward>)
tensor(0.2201, grad_fn=<MseLossBackward>)
tensor(1.6965, grad_fn=<MseLossBackward>)
tensor(2.2674, grad_fn=<MseLossBackward>)
tensor(0.8503, grad_fn=<MseLossBackward>)
tensor(3.9400, grad_fn=<MseLossBac

tensor(2.1493, grad_fn=<MseLossBackward>)
tensor(1.4721, grad_fn=<MseLossBackward>)
tensor(0.3556, grad_fn=<MseLossBackward>)
tensor(0.0686, grad_fn=<MseLossBackward>)
tensor(0.8190, grad_fn=<MseLossBackward>)
tensor(1.3189, grad_fn=<MseLossBackward>)
tensor(0.5415, grad_fn=<MseLossBackward>)
tensor(2.1185, grad_fn=<MseLossBackward>)
tensor(0.2484, grad_fn=<MseLossBackward>)
tensor(0.2557, grad_fn=<MseLossBackward>)
tensor(1.6299, grad_fn=<MseLossBackward>)
tensor(0.2611, grad_fn=<MseLossBackward>)
tensor(0.2081, grad_fn=<MseLossBackward>)
tensor(1.6334, grad_fn=<MseLossBackward>)
tensor(1.4812, grad_fn=<MseLossBackward>)
tensor(1.1527, grad_fn=<MseLossBackward>)
tensor(1.1360, grad_fn=<MseLossBackward>)
tensor(1.8345, grad_fn=<MseLossBackward>)
tensor(1.3698, grad_fn=<MseLossBackward>)
tensor(0.2622, grad_fn=<MseLossBackward>)
tensor(0.6743, grad_fn=<MseLossBackward>)
tensor(0.7740, grad_fn=<MseLossBackward>)
tensor(2.2804, grad_fn=<MseLossBackward>)
tensor(1.7446, grad_fn=<MseLossBac

KeyboardInterrupt: 

In [ ]:
import pdb
class Network(object):
    def __init__(self,size):
        self.model = torch.nn.Sequential(
            torch.nn.Linear(size[0], size[1], bias = False), # w_1 * x +b_1
            torch.nn.ReLU(),
            torch.nn.Linear(size[1], size[2] , bias = False),
            torch.nn.ReLU(),
            torch.nn.Linear(size[2], size[3] , bias = False),
            torch.nn.Sigmoid(),
            device = device
        )
        self.loss_fn = nn.MSELoss(reduction = 'sum')
        
    def run(self,training_data,epochs,mini_batch_size,eta,test_data = None):
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = eta)
        
        if test_data: 
            test_data = list(test_data)
            test_data = test_data[:1000]
            n_test = len(test_data)
        training_data = list(training_data)
        
#         training_data = training_data[:1000] #可以改变训练集大小，测试使用
        
        n = len(training_data)
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            k = len(mini_batches)
            for  mini_batch in mini_batches:
                loss = self.training(mini_batch,mini_batch_size)
            print('LOSS {0},'.format(loss))
            if j >= 0:
                print('Epoch {0}:{1} / {2}'. format(
                        j, self.test(test_data), n_test))
    def training(self, training_data, size):
        x = torch.zeros((size,784),dtype = torch.float).to(device)
        y = torch.zeros((size,10), dtype = torch.float).to(device)
        for (a, b), i in zip(training_data, range(size)):
            x[i] = torch.from_numpy(a).to(device) #转换成二维的矩阵，后面做乘法方便
            y[i] = torch.from_numpy(b).to(device)
        #Forward pass
        y_pred = self.model(x) # model.forward()
        #loss
        loss = self.loss_fn(y_pred , y) #computation graph  
        
        #重置参数增加量
        self.optimizer.zero_grad()
        # Backward pass
        loss.backward()
            
        #update model parameters
        self.optimizer.step()
        
        return loss.item()
    def test(self,test_data):
        test_results = [(torch.argmax(self.model(torch.from_numpy(x).reshape(1,784).float().to(device))), torch.from_numpy(y).to(device))
                for x, y in test_data]
        return sum(y[int(x)] == 1 for (x, y) in test_results)